### . Código completo do Bubi Chatbot

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

llm= ChatGoogleGenerativeAI(
    model="gemini-2.5-pro", api_key=os.getenv("GOOGLE_API_KEY")
)

### Prompt + LLMChain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template="Explique de forma clara o produto chamado {produto}. em ate {palavras} palavras."

prompt= PromptTemplate.from_template(template, partial_variables={"palavras": 50})

chain= LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True 
)

In [ ]:
resposta= chain.invoke({"produto": "intel vpro enteprise"})


In [ ]:
print("Bubi:", resposta["text"])

### SequentialChain (explicação + resumo)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

template="Explique de forma clara o produto chamado {produto}. em ate {palavras} palavras."

prompt= PromptTemplate.from_template(template, partial_variables={"palavras": 50},)

chain= LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True ,
    output_key="Definição do Produto"
)

template2=" Crie um titulo legal para o produto {produto} coloque as pricipais caracteritica e preços em até {palavras} palavras."

prompt2= PromptTemplate.from_template(template2, partial_variables={"palavras": 50})

chain2= LLMChain(
    llm=llm,
    prompt=prompt2,
    verbose=True ,
    output_key="caracteristica do Produto"
)

In [ ]:
chainSeq = SequentialChain (
    chains=[chain, chain2],
    input_variables=["produto"],
    output_variables=["Definição do Produto", "caracteristica do Produto"],
    verbose=True
)

In [ ]:
result = chainSeq.invoke({"produto": "Mouse Gamer 7200DPI"})


In [ ]:
result["Definição do Produto"]

In [ ]:
print(result["caracteristica do Produto"])

## 5. RAG com FAISS (FAQ + catálogo)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd

faqLoad=TextLoader("../data/faq.txt", encoding="utf-8")

#Carrega produtos CSV e converte para texto
df = pd.read_csv("dados/produtos.csv")
with open("dados/produtos.txt", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        f.write(f"{row['nome']}: {row['descricao']}, R$ {row['preco']}\n")

prod_loader = TextLoader("dados/produtos.txt", encoding="utf-8")
prod_docs = prod_loader.load()